In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.2. Ordinal Regression/Output')
cur_date = "053124"

library(readxl)
library(openxlsx)
library(tidyverse)
library(ordinalNet)
library(reshape2)

# reading in file
biomarker_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 2)) # GO BACK FOR IMPUTED VALUES!!!
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
mediator_scores_df = data.frame(read_excel("Input/Mediator_Scores_BP_Subjects_052724.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message:
“Expecting numeric in D1077 / R1077C4: got 'NA'”
Warning message:
“Expecting numeric in D1080 / R1080C4: got 'NA'”
Warning message:
“Expecting numeric in D1083 / R1083C4: got 'NA'”
Warning message:
“Expecting numeric in D1091 / R1091C4: got 'NA'”
Warning message:
“Expecting numeric in D1097 / R1097C4: got 'NA'”
Warning message:
“Expecting numeric in D1098 /

In [2]:
head(bp_df)
head(subject_info_df)
head(mediator_scores_df)

,Subject_ID,Systolic,Diastolic,BP_Classification
,<chr>,<dbl>,<dbl>,<chr>
1,CS_26,133,86,1
2,CS_29,116,84,N
3,CS_17,118,80,N
4,NS_63,122,85,E
5,CS_21,112,77,N
6,CS_27,131,91,1


,Study,Original_Subject_Number,Subject_Number,Group,Subject_ID,Sex,Age,Race
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,FM2,97,1,CS,CS_1,M,18,B
2,FM2,99,2,CS,CS_2,F,20,W
3,FM2,101,3,CS,CS_3,F,20,W
4,FM2,98,4,CS,CS_4,F,21,W
5,TCORS LAIV,39,5,CS,CS_5,M,21,W
6,FM2,100,6,CS,CS_6,F,22,W


,Variable,Subject_ID,Mediator_Score
,<chr>,<chr>,<dbl>
1,Noradrenaline,CS_5,0.9209663
2,Noradrenaline,CS_8,0.5018144
3,Noradrenaline,CS_10,0.8466564
4,Noradrenaline,CS_16,0.5018144
5,Noradrenaline,CS_17,0.3297612
6,Noradrenaline,CS_21,0.2711344


In this analysis, we're using [ordinal regression](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/ordinal-regression/) to predict blood pressure classification using mediator scores from 17 biomarkers. 

In [3]:
# creating a vector for variables that increase atherosclerotic risk
# HDL decreases risk
#allostatic_load_biomarkers = c('Cortisol','Noradrenaline','Epinephrine','Hba1c','Fibrinogen','CRP', 'HDL')

# creating 1 df
ordinal_regression_df = inner_join(bp_df[,c(1,4)], subject_info_df[,4:8]) %>%
    # adding allostatic load data
    inner_join(mediator_scores_df) %>%
    # obtaining allostatic load biomarkers only
    #filter(Variable %in% allostatic_load_biomarkers) %>%
    pivot_wider(names_from = Variable, values_from = Mediator_Score)

# turning BP into a factor
ordinal_regression_df$BP_Classification = factor(ordinal_regression_df$BP_Classification, levels = c("N", "E", "1", '2'))

#reordering columns
ordinal_regression_df = ordinal_regression_df[,c(1,3:23,2)]
    

head(ordinal_regression_df)

Joining with `by = join_by(Subject_ID)`
Joining with `by = join_by(Subject_ID)`


Subject_ID,Group,Sex,Age,Race,Noradrenaline,Hba1c,Fibrinogen,CRP,IP10,⋯,IL13,IL1B,IL2,IL4,IL6,IL8,TNFa,HDL,Epinephrine,BP_Classification
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
CS_26,CS,F,34,B,0.00000000,0.1261076,0.35238170,0.03189947,0.07017161,⋯,0.3271404,0.00000000,2.351685e-07,0.25823172,0.18675526,0.001686759,0.14780720,0.01961938,0.09668427,1
CS_29,CS,F,36,B,0.09028609,0.3276962,0.18618128,0.04754286,0.03571227,⋯,0.7220417,0.01153174,1.419332e-08,0.53046857,0.43704343,0.355268670,0.08243793,0.29810436,0.48260060,N
CS_17,CS,M,29,W,0.32976117,0.2873836,0.18107212,0.01664812,0.12055087,⋯,0.3411684,0.00000000,6.761111e-08,0.49508420,0.03189408,0.147350214,0.26384236,0.52779975,0.72404411,N
NS_63,NS,M,43,B,0.25782803,0.4462655,0.11215630,0.22912585,0.42703285,⋯,0.6317240,0.00000000,1.132208e-06,0.60115086,1.00000000,0.119470619,0.42067290,0.13049306,0.67048546,E
CS_21,CS,F,31,B,0.27113435,0.7550424,0.16931582,0.12987698,0.35359516,⋯,0.2035670,0.00000000,9.701944e-07,0.07874693,0.30264289,0.418614119,0.26384236,0.06673031,0.31548045,N
CS_27,CS,F,35,W,0.23404945,0.3807725,0.03079665,0.02882542,0.02232779,⋯,0.4987997,0.00000000,7.491238e-08,0.35320369,0.14274251,0.033546943,0.38209701,0.14858426,0.00000000,1


In [4]:
# creating a vector of booleans that specifies whether or not the coefficient should be
# positive (TRUE) or negative (FALSE), only HDL will be negative
biomarker_coefficients = c(rep(TRUE, ncol(ordinal_regression_df) - 8), FALSE, TRUE)

estimateOrdWeights <- function(dataset, response, coefficients_magnitude){
    # ADD WORDS
    # ordinal regression or estimating ordinal weights
    
    ############# get required Regression Parameters
    
    #get number of response levels
    NoBins <- length(unique(dataset[[response]]))

    #get number of features or slieces
    noSlices <- ncol(dataset) - 2
    
    #run the regression
    feature_data <- as.matrix(dataset[,c(2:(ncol(dataset)-1))])
    #class(featuredata) <- "numeric"

    # ordinal regression
    # positiveID specifies whether or not each coefficient for each ind. variable should be constrained to be non-negative
    regression_model <- ordinalNet(feature_data, dataset[[response]], positiveID = coefficients_magnitude, 
                            #lambdaVals defines how quickly your coefficients drop to 0
                            # keeping it at 0 tries to minimize feature reduction
                            # maxiterout defines the number of iterations before it stops
                         lambdaVals = 0, maxiterOut = 100)
    testW <- regression_model$coefs[NoBins:(noSlices + NoBins - 1)]

    # rescaling the coefficients so that they sum to 1
    normalizer = 1/sum(testW)
    rescaled_weight <- testW * normalizer

    # adding the variable names back in 
    test_weights_df = data.frame(Variable = colnames(feature_data), Coefficient = rescaled_weight) %>%
        arrange(-Coefficient) 
    
    return(test_weights_df)
}

# calling fn
biomarker_coefficients_df = estimateOrdWeights(ordinal_regression_df[,c(1,6:23)], 'BP_Classification', biomarker_coefficients)

head(biomarker_coefficients_df)

,Variable,Coefficient
,<chr>,<dbl>
1,IL2,0.35626845
2,IL8,0.17808930
3,IL12p70,0.14744960
4,HDL,0.09240571
5,IL10,0.08858715
6,Epinephrine,0.05554931


Why wasn't HDL able to be constrained to negative?
What do the values mean? What considered high vs. low?
How does this first function relate to the second function below?

Run a linear regression to determine allostatic load score before looking at the chunk below. This will allow me to focus on the known samples and rank those using the code

# Linear Regression

Now that we have these weights from the ordinal regression, we'll use a linear regression to calculate allostatic load scores for each subject based on their raw biomarker sample values.

First, we'll combine our data.

In [5]:
head(biomarker_df)

,Subject_ID,Category,Variable,Value
,<chr>,<chr>,<chr>,<dbl>
1,CS_1,AL Biomarker,Cortisol,140.304
2,CS_2,AL Biomarker,Cortisol,246.542
3,CS_3,AL Biomarker,Cortisol,50.878
4,CS_4,AL Biomarker,Cortisol,137.387
5,CS_5,AL Biomarker,Cortisol,116.602
6,CS_6,AL Biomarker,Cortisol,54.030


In [14]:
wider_biomarker_df = dcast(biomarker_df, Variable ~ Subject_ID, value.var = "Value") %>% 
    inner_join(biomarker_coefficients_df) %>%
    column_to_rownames("Variable")

head(wider_biomarker_df)

Joining with `by = join_by(Variable)`


,CS_1,CS_10,CS_11,CS_12,CS_13,CS_14,CS_15,CS_16,CS_17,CS_18,⋯,NS_55,NS_56,NS_57,NS_58,NS_59,NS_60,NS_61,NS_62,NS_63,Coefficient
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CRP,5.559845e+04,2.863647e+06,77646.97829,2.080247e+05,4.750068e+06,3478791.4520,5.037985e+06,1.296296e+06,8.458425e+05,4.158242e+06,⋯,7.252398e+06,6.516544e+06,4.517126e+06,9.830317e+04,2.782011e+07,1.560861e+07,1.734071e+07,4.514378e+07,1.055221e+07,0.00000000
Epinephrine,5.405000e+00,6.495000e+00,NA,4.843000e+00,4.221000e+00,5.3940,8.817000e+00,1.094700e+01,9.020000e+00,7.161000e+00,⋯,3.842000e+00,NA,9.262000e+00,5.655000e+00,6.193000e+00,NA,8.111000e+00,5.844000e+00,8.646000e+00,0.05554931
Fibrinogen,9.448052e+05,1.177711e+06,355916.01900,2.570534e+06,7.491614e+05,290255.0780,3.788870e+05,3.866305e+05,9.346629e+05,9.929176e+05,⋯,9.841312e+05,1.045044e+06,1.633222e+06,8.622901e+05,1.027264e+06,8.765986e+05,2.277013e+06,1.303893e+06,7.260825e+05,0.00000000
Hba1c,3.000829e+03,2.033487e+04,199.15000,9.529201e+03,1.718140e+02,611.6580,6.109290e+02,9.741940e+03,1.324357e+04,4.094500e+02,⋯,4.485130e+02,1.418739e+04,1.572274e+04,1.190690e+04,2.151225e+04,1.069195e+04,7.531294e+03,2.102216e+04,1.640163e+04,0.00000000
HDL,2.342104e+03,6.781930e+02,1638.45200,3.487302e+03,2.103010e+03,1945.3195,1.468414e+03,1.164451e+03,2.309097e+03,2.216117e+03,⋯,2.639136e+03,8.904290e+02,1.518944e+03,1.357090e+03,1.158884e+03,1.261489e+03,2.226878e+03,1.026427e+03,1.020604e+03,0.09240571
IFNg,8.934312e+00,1.641966e+00,20.58531,1.814071e+01,2.501686e+01,16.6021,2.269073e+01,1.907438e+01,3.998119e+00,1.373203e+01,⋯,7.950224e+00,5.419301e+00,3.051331e+00,1.893192e+00,1.058480e+01,4.823686e+00,5.547756e+00,1.253785e+01,4.678406e+00,0.00000000


In [55]:
# words
multiplication_df = wider_biomarker_df[,1:63] %>%
    # words
    mutate(across(contains("_"), ~.x*wider_biomarker_df$Coefficient))

# keeping rows with only zeros ADD MORE WORDS
zero_booleans = apply(multiplication_df, 1, function(row) all(row == 0)) 
zero_rownames = rownames(multiplication_df)[zero_booleans]

`%notin%` <- Negate(`%in%`)
zero_df = wider_biomarker_df[rownames(wider_biomarker_df) %in% zero_rownames,] %>%
    select(-Coefficient)
non_zero_df = multiplication_df[rownames(multiplication_df) %notin% zero_rownames,] 
                          

# recombining data
weighted_mediator_score_df = rbind(zero_df, non_zero_df)
                      
head(weighted_mediator_score_df)

,CS_1,CS_10,CS_11,CS_12,CS_13,CS_14,CS_15,CS_16,CS_17,CS_18,⋯,NS_54,NS_55,NS_56,NS_57,NS_58,NS_59,NS_60,NS_61,NS_62,NS_63
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CRP,5.559845e+04,2.863647e+06,7.764698e+04,2.080247e+05,4.750068e+06,3.478791e+06,5.037985e+06,1.296296e+06,8.458425e+05,4.158242e+06,⋯,2.567619e+05,7.252398e+06,6.516544e+06,4.517126e+06,9.830317e+04,2.782011e+07,1.560861e+07,1.734071e+07,4.514378e+07,1.055221e+07
Fibrinogen,9.448052e+05,1.177711e+06,3.559160e+05,2.570534e+06,7.491614e+05,2.902551e+05,3.788870e+05,3.866305e+05,9.346629e+05,9.929176e+05,⋯,6.912422e+05,9.841312e+05,1.045044e+06,1.633222e+06,8.622901e+05,1.027264e+06,8.765986e+05,2.277013e+06,1.303893e+06,7.260825e+05
Hba1c,3.000829e+03,2.033487e+04,1.991500e+02,9.529201e+03,1.718140e+02,6.116580e+02,6.109290e+02,9.741940e+03,1.324357e+04,4.094500e+02,⋯,1.341513e+04,4.485130e+02,1.418739e+04,1.572274e+04,1.190690e+04,2.151225e+04,1.069195e+04,7.531294e+03,2.102216e+04,1.640163e+04
IFNg,8.934312e+00,1.641966e+00,2.058531e+01,1.814071e+01,2.501686e+01,1.660210e+01,2.269073e+01,1.907438e+01,3.998119e+00,1.373203e+01,⋯,3.802989e+00,7.950224e+00,5.419301e+00,3.051331e+00,1.893192e+00,1.058480e+01,4.823686e+00,5.547756e+00,1.253785e+01,4.678406e+00
IL1B,1.039313e-01,6.975240e-04,8.246930e-02,1.764078e-01,1.318969e-01,7.718695e-02,1.038359e-01,6.975240e-04,6.975240e-04,4.349375e+01,⋯,6.975240e-04,1.189953e-01,6.975240e-04,3.875969e-02,6.975240e-04,6.975240e-04,6.975240e-04,6.975240e-04,6.975240e-04,6.975240e-04
IL6,9.701685e-01,3.002960e-01,7.664941e-01,3.081771e+00,2.799783e+00,2.562255e+00,3.855867e+00,5.057684e-01,1.185886e-01,2.196781e+00,⋯,1.653713e-01,3.556114e+00,8.178470e-01,2.757633e-01,5.929432e-02,1.377326e+00,2.905032e-01,8.448595e-01,1.238455e+00,1.918395e+00


In [60]:
# calculating allostatic load by summing by the columns
allostatic_load_df = data.frame(Allostatic_Load = apply(weighted_mediator_score_df, 2, sum)) %>%
    arrange(-Allostatic_Load)
allostatic_load_df

,Allostatic_Load
,<dbl>
NS_61,71204937
NS_53,46888864
NS_62,46473287
CS_4,37430857
NS_41,33182434
NS_59,28869120
CS_32,20822562
CS_33,13356327
NS_63,11296798
